In [2]:
import ee
ee.Initialize()


In [4]:
import ee
import folium
from folium.plugins import HeatMap


In [19]:
# Define an area of interest (Sydney, Australia, including the Blue Mountains and Warrimoo)
aoi = ee.Geometry.Rectangle([150.5, -33.9, 151.5, -33.3])

# Load a Landsat 8 image collection (Collection 2) filtered by date and location
landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2018-03-01', '2018-03-31') \
    .sort('CLOUD_COVER') \
    .first()

# Check if an image is found
if landsat_collection:
    print("Landsat image from Collection 2 found and loaded.")
else:
    print("No image found for the specified date and location.")


Landsat image from Collection 2 found and loaded.


In [20]:
# Calculate NDVI using updated band names for Collection 2
ndvi = landsat_collection.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')


In [21]:
# Function to apply scaling factors to Landsat Collection 2 data
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_band = image.select('ST_B10').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_band, None, True)

# Apply the scaling factors
scaled_landsat = apply_scale_factors(landsat_collection)

# Calculate NDVI on the scaled image
ndvi = scaled_landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')


In [22]:
# Define visualization parameters for NDVI
ndvi_params = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'green']
}

# Create a folium map object centered at the specified location
map_center = [-33.7, 151.0]  # Centered around Sydney
my_map = folium.Map(location=map_center, zoom_start=10)

# Add the NDVI layer to the map
ndvi_layer = folium.raster_layers.TileLayer(
    tiles=ndvi.getMapId(ndvi_params)['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    name='NDVI',
    overlay=True,
    control=True
)

ndvi_layer.add_to(my_map)

# Add layer control to toggle layers
folium.LayerControl().add_to(my_map)

# Display the map
my_map
